In [ ]:
#Paquetes de instalacion
'''!conda install -c anaconda pandas
!conda install -c anaconda cloudpickle
!conda install -c anaconda numpy
!conda install -c conda-forge matplotlib
!conda install -c anaconda seaborn
!conda install -c anaconda scipy
!conda install -c conda-forge python-pdfkit
!conda install -c anaconda statsmodels
!conda install -c anaconda xlrd'''

In [ ]:
from sklearn import preprocessing
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os,inspect, gc
import sys
import re
import math

In [ ]:
path_file = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path_file=os.path.join(path_file,'datos_reales')

# 1. Modulos de lectura y guardado de archivos csv y xlsx

In [ ]:
#excelFile(file)
    #retorna xlsx file, util para archivos excel con multiples pestañas
#parameters:
    #file: string
           #nombre del archivo excel
def excelFile(file):
    return pd.ExcelFile(open(os.path.join(path_file,file), 'rb'))

In [ ]:
#readFile(file, sheet=None)
    #retorna data frame
#parameters:
    #file: string
        #nombre del archivo excel o csv
    #sheet: String
        #nombre de una pestaña en la hoja excel
def readFile(file, sheet=None):
    if file.endswith('.xlsx') or file.endswith('.xls'):
        if sheet == None:
            raise Exception('sheet name should be defined')
        else:
            return pd.read_excel(open(os.path.join(path_file,file), 'rb'), sheet_name=sheet)
    elif file.endswith('.csv'):
        return pd.read_csv(os.path.join(path_file,file))
    else:
        raise Exception('Types would be 0 or 1')

In [ ]:
#saveFile(dataFrame, file, sheet='hoja1')
    #Guarda un data frame en un archivo excel o csv
#parameters:
    #dataFrame: dataFrame
        #data frame a guardar
    #out: string
        #nombre del resultado xlsx o csv
    #sheet: String
        #nombre de una pestaña en la hoja excel
def saveFile(dataFrame, out, sheet='hoja1'):
    if out.endswith('.xlsx') or out.endswith('.xls'):
        if sheet == None:
            raise Exception('sheet name should be gived')
        else:
            dataFrame.to_excel("datos_reales/"+out, sheet_name=sheet)
    elif out.endswith('.csv'):
        dataFrame.to_csv("datos_reales/"+out)
    else:
        raise Exception('Support only .xlsx, .xls, .csv files')

# 2. Preprocesamiento

In [ ]:
#splitByColumn(dataFrame, by, out)
        #separa un dataFrame mediante grupos y retorna o guarda cada grupo en un excel segun frame
#Parameters:
    #dataFrame: dataFrame
        #dataframe a separar y guardar
    #by: String
        #columna mediante el cual generar grupos
    #out: String
        #nombre del resultado xlsx, por defecto nombre de la funcion
    #frame: Boolean
        #Si es true retorna el dataframe agrupado mediante by, caso contrario, guarda el dataframe agrupado por grupos en un solo excel
def splitByColumn(dataFrame, by, out="splitByColumn.xlsx", frame=False):
    if frame:
        return dataFrame.groupby(by=by)
    else:
        with pd.ExcelWriter('datos_reales/'+out) as writer:
            for key, group in dataFrame.groupby(by=by):
                group.reset_index(drop=True).to_excel(writer, sheet_name=key, index=True)
            
#data=readFile('hoja1_WDIEXCEL.csv')
#splitByColumn(data, "Country Code", frame=False)

In [ ]:
#def countMissingDataPerYear(dataFrame, initYear, endYear)
    # retorna un dataFrame, cuenta el numero de datos faltantes (nan y cero) por año desde un año inicial hasta uno final (años en las columnas)
#Parameters:
    #datFrame: Dataframe
        #dataframe a analizar
    #initYear: Int
        #Año inicial
    #endYear: Int
        #Año final
def countMissingDataPerYear(dataFrame, initYear, endYear):
    dataFrame = dataFrame.replace(0, np.nan)
    columns = list(dataFrame.columns)
    init_year = columns.index(str(initYear))
    end_year = columns.index(str(endYear))
    total = dataFrame.shape[0]
    data_frame_year = dataFrame.iloc[:,init_year:end_year+1]
    count_nan = data_frame_year.isna().sum(axis=0)
    percent_nan = (100 * count_nan)/total
    data_frame = pd.DataFrame(count_nan, columns=['# Nan values'])
    data_frame["100%"] = total
    data_frame["Missing%"] = percent_nan
    return data_frame
#data=readFile('splitByColumn.xlsx', "BRA")
#data = countMissingDataPerYear(data,1960, 2018)
#saveFile(data, "countMissingDataPerYear.xlsx", sheet='BRA')

In [ ]:
#def countMissingDataPerIndicator(dataFrame, initYear, endYear)
    # retorna un dataFrame, cuenta el numero de datos faltantes(ceros y nan) por indicador desde un año inicial hasta uno final (años en las columnas)
#Parameters:
    #datFrame: Dataframe
        #dataframe a analizar
    #initYear: Int
        #Año inicial
    #endYear: Int
        #Año final
def countMissingDataPerIndicator(dataFrame, initYear, endYear):
    dataFrame = dataFrame.replace(0, np.nan)
    columns = list(dataFrame.columns)
    init_year = columns.index(str(initYear))
    end_year = columns.index(str(endYear))
    data_frame_year = dataFrame.iloc[:,init_year:end_year+1]
    total = data_frame_year.shape[1]
    count_nan = data_frame_year.isna().sum(axis=1)
    percent_nan = (100 * count_nan)/total
    data_frame = pd.DataFrame(count_nan, columns=['# Nan values'])
    data_frame["100%"] = total
    data_frame["Missing%"] = percent_nan
    data_frame.index = dataFrame["Indicator Code"]
    return data_frame.sort_values(by="Missing%", ascending=False)
#data=readFile('splitByColumn.xlsx', "BRA")
#data = countMissingDataPerIndicator(data,1990, 2017)
#saveFile(data, "countMissingDataPerIndicator.xlsx", sheet='hoja1')


### DataFrame 
Sera utilizado para todo el analisis, contiene unicamente años e indicadores

In [ ]:
#dataFrame(dataFrame, initYear, endYear, transpose=False)
    #Retorna dataFrame entre años especificados, unicamente contiene años e indicadores y se remplaza todo los valores 0 con NAN
#Parameters:
    #dataFrame: DataFrame
    #initYear: Int
        #año desde comenzara el data frame
    #endYear: Int
        #año donde concluira el data frame
    #transpose: Bool
        #Si es true retorna un dataFrame[index(year), columns(indicators)] caso contrario dataFrame[index(indicators), columns(year)]
def dataFrame(dataFrame, initYear, endYear, transpose=True):
    dataFrame = dataFrame.replace(0, np.nan)
    columns = list(dataFrame.columns)
    init_year = columns.index(str(initYear))
    end_year = columns.index(str(endYear))
    data_frame_year = dataFrame.iloc[:,init_year:end_year+1]
    data_frame_year.index = dataFrame["Indicator Code"]
    if transpose:
        data_frame_year = data_frame_year.T
        #newdata = newdata.reset_index()
    return data_frame_year

#data = readFile('splitByColumn.xlsx',"BRA")
#data= dataFrame(data, 1990,2017)
#data.head(10)

### Ind. <= x% 
porcentaje permitido de datos faltantes $Ind<p\%$ en un indicador<br>
$\huge umbral=\frac{(numAños * P\%)}{100}$ <br>

In [ ]:
#delIndicatorMayorUmbralZerosOrNan(dataFrame, p=0)
    #retorna dataFrame eliminando aquellas columnas que contienen 0 or Nan mayor o igual que el umbral 
#Parameters:
    #dataFrame: DataFrame
        #dataframe con indicadores en las columnas y años en los indices
    #p: int entre 0 y 100
        # define el numero de ceros o nan permitidos
def delIndicatorMayorUmbralZerosOrNan(dataFrame, p=0):
    years = dataFrame.shape[0]
    umbral = round((years*p)/100)
    #data_frame = (dataFrame==0).astype(int)
    data_frame = dataFrame.loc[:,dataFrame.isna().sum(axis=0)<umbral]
    data_frame.index.name="year"
    return data_frame


#data = readFile('splitByColumn.xlsx',"BRA")
#data= dataFrame(data, 1990,2017)
#data=delIndicatorMayorUmbralZerosOrNan(data,25)
#saveFile(data, "delIndicatorMayorUmbralZerosOrNan.xlsx", sheet='BRA')

In [ ]:
#contrast = lambda x: True if (x.replace(0,np.nan).dropna().values == x.replace(0,np.nan).dropna().values[0]).all() else False
    #Remplaza los ceros por NAN y luego los elimina, luego compara toda la lista de valores con su valor en la posicion 0
    #Retorna los indicadores sin cambios en el tiempo
#def delIndicatorWithoutChanges(dataFrame, opcion = True)
    #retorna dataFrame
#Parameters:
    #dataFrame: DataFrame
    #opcion: Bool
        #Si es True retorna los indicadores constantes, caso contrario retorna el dataFrame sin incluir los valores constantes
contrast = lambda x: True if (x.replace(0,np.nan).dropna().values == x.replace(0,np.nan).dropna().values[0]).all() else False
def delIndicatorWithoutChanges(dataFrame, opcion = True):
    data_frame = dataFrame.set_index("year")
    booleans = data_frame.apply(contrast)
    if opcion:
        trues  = booleans[booleans]
        return data_frame.loc[:,trues.index]
    else:
        booleans = np.invert(booleans)
        trues  = booleans[booleans]
        return data_frame.loc[:,trues.index]


#data = readFile('delIndicatorMayorUmbralZerosOrNan.xlsx',"BRA")
#data = delIndicatorWithoutChanges(data,False)
#saveFile(data, "delIndicatorWithoutChanges.xlsx", sheet='BRA')

### Outlier

In [ ]:
#outlier(dataFrame)
    #Retorna dos DataFrame(sinAtipicos, soloAtipicos), ademas elimina aquellas columnas que al eliminar el outlier resulta ser todo vacio
#Parameters:
    #dataFrame:DataFrame
def outlier(dataFrame):
    data_frame = dataFrame.replace(0,np.nan)
    data_frame_outlier = data_frame.copy()
    quantiles = data_frame.quantile([0.25, 0.5, 0.75], axis=0)
    iqr = quantiles.iloc[2,:] - quantiles.iloc[0,:]
    minimun = quantiles.iloc[0,:] - 1.5*iqr
    maximun =  quantiles.iloc[2,:] + 1.5*iqr
    #dataframe sin valores atipicos
    data_frame = data_frame[data_frame>minimun]
    data_frame = data_frame[data_frame<maximun]
    data_frame = data_frame.dropna(axis=1, how="all")
    trues = [col for col in data_frame_outlier.columns if col not in data_frame.columns]
    #dataframe con valores atipicos
    data_frame_outlier = data_frame_outlier.loc[:,trues]
    #data_frame_outlier = data_frame_outlier[data_frame_outlier<minimun]
    
    return data_frame, data_frame_outlier

#data=readFile("delIndicatorWithoutChanges.xlsx", "BRA").set_index("year")
#data, data_atipicos = outlier(data)
#saveFile(data, "outlier.xlsx", sheet='BRA')

### correlation coefficient
Este coeficiente sólo debe utilizarse para comparar variables cuantitativas y continuas<br>
$\huge r=\frac{\sum_{i=1}^{n}(x_{i}-\bar{x})(y_{i}-\bar{y})}{\sqrt{\sum_{i=1}^{n}(x_{i}-\bar{x})^{2}\sum_{i=1}^{n}(y_{i}-\bar{y})^{2}}}$ <br>
el rango varia entre [-1,1] donde: 1 indica un alto grado de correlacion directa, -1 un alto grado de correlacion inversa y 0 no
existe correlacion lineal

In [ ]:
#joinSeriesAndWdi(series, wdi)
    #retorna dataFrame, se realiza union entre dos dataFrames
#Parameters:
    #series: DataFrame
        #dataframe con columnas de series code, topicos, subtopicos (seriesCETS.csv)
    #wdi: DataFrame
        #dataframe resultado de outlier
def joinSeriesAndWdi(series, wdi):
    wdi = wdi.set_index("year").T
    wdi.index.name = "Indicator Code"
    wdi = wdi.reset_index()
    inner_join = pd.merge(left=series, right=wdi, left_on="Series Code", right_on="Indicator Code", how="inner")
    inner_join = inner_join.drop(columns="Series Code").set_index("Indicator Code")
    inner_join.index.name = "Indicator Code"
    return inner_join

#series=readFile("seriesCETS.csv")
#wdi = readFile("outlier.xlsx", "BRA")
#saveFile(joinSeriesAndWdi(series,wdi),"joinSeriesAndWdi.xlsx", sheet='BRA')

In [ ]:
#correlation(dataFrame, plot = False)
    #Retorna dataFrame(correlacion de pearson)
#Parameters:
    #dataFrame: DataFrame
    #plot: Bool
        #si es true grafica mediante un mapa de calor la correlacion entre indicadores
def correlation(dataFrame, plot = False):
    corr = dataFrame.corr(method="pearson")
    corr.index.name = 'index'
    if plot:
        mask = np.zeros_like(corr)
        mask[np.triu_indices_from(mask)] = True
        with sns.axes_style("white"):
            plt.figure(figsize=(10,10))
            ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True, annot=True)
            plt.show()
    return corr
#============No tocar esta seccion=====================

#data=readFile("outlier.xlsx", "BRA").set_index("year")
#data = correlation(data)
#saveFile(data, "corr.xlsx", sheet='hoja1')

#runCorrelation(nameMultiSheetExcel, out = "corr_multiple.xlsx", sheets = [])
    #guarda la correlacion en varias hojas de un archivo excel
#Parameters:
    #nameMultiSheetExcel:String or group
        #nombre del archivo excel con multiples hojas #series=readFile("joinSeriesAndWdi.xlsx", sheet="BRA")->splitByColumn(series, by="Topic", out="topic_BRA.xlsx", frame=False)
        #o un dataFrame agrupado splitByColumn(series, by="Topic", out="topic_BRA.xlsx", frame=True)
    #out:String
        #output name con extencion xlsx
    #sheets:Array[String]
        #Nombre de las hojas en el archivo excel
def runCorrelation(nameMultiSheetExcel, out = "corr_multiple.xlsx", sheets = []):
    if isinstance(nameMultiSheetExcel, str):
        xlsx = excelFile(nameMultiSheetExcel)
        with pd.ExcelWriter('datos_reales/{}'.format(out)) as writer:
            for sheet in sheets:
                sector = pd.read_excel(xlsx, sheet_name="{0}".format(sheet)).reset_index(drop=True).set_index('Indicator Code')
                #Se establece en 5 puesto que los años en pathMultiSheetExcel empiezan en este punto
                corr = correlation(sector.iloc[:,5:].T)
                corr.to_excel(writer, sheet_name=sheet, index=True)
    else:
        with pd.ExcelWriter('datos_reales/{}'.format(out)) as writer:
            for key, group in nameMultiSheetExcel:
                sector = group.reset_index(drop=True).set_index('Indicator Code')
                corr = correlation(sector.iloc[:,5:].T)
                corr.to_excel(writer, sheet_name=str(key), index=True)
                
#series=readFile("joinSeriesAndWdi.xlsx", sheet="BRA")
#splitByColumn(series, by="Topic", out="topic_BRA.xlsx")
#nameMultiSheetExcel = "topic_BRA.xlsx"
#runCorrelation(nameMultiSheetExcel, out = "corr_multiple.xlsx", sheets = ['Education','Environment', "Economic Policy & Debt",
#                                                                       "Financial Sector", "Health", "Infrastructure", 
#                                                                          "Poverty", "Private Sector & Trade", "Public Sector", "Social Protection & Labor"])

In [ ]:
#getCorreatedVariables(matrixCorrelated, threshold, name=True)
    #retorna dataFrame de correlaciones que sean mayores o iguales a threshold (dataFrame=['X','Y','Val'])
#Parameters:
    #matrixCorrelated: DataFrame
        #Matriz de correlacion
    #threshold: Float
        #umbral minimo fijado
    #names: Bool
        #si es true retorna ['X'=Indicator,'Y'=Indicator,'Val'=Correlacion] caso contrario ['X'=i,'Y'=j,'Val'=Correlacion]
def getCorreatedVariables(matrixCorrelated, threshold, name=True):
    ones = np.ones(matrixCorrelated.shape)
    above_diag = np.triu(ones, k=1)#get data only above to diag, and the rest push 0 (), considera los datos encima de la diagonal
    get_index = np.where(above_diag)
    #(nombreIndice,nombreColumn,valor>=threshold)
    if name:
        variables_correlated = [(matrixCorrelated.index[i],matrixCorrelated.columns[j], matrixCorrelated.iat[i,j]) for i,j in zip(*get_index) if np.abs(matrixCorrelated.iat[i,j])>= threshold]
    else:
    #(posicionIndice, posicionColumn, valor>=threshold)
        variables_correlated = [(i,j, matrixCorrelated.iat[i,j]) for i,j in zip(*get_index) if np.abs(matrixCorrelated.iat[i,j])>= threshold]
    data_frame = pd.DataFrame(variables_correlated, columns=['X','Y','Val'])
    return data_frame

In [ ]:
def numberIndicatorsBySector(group):
    return group.agg(Total=("Indicator Code","count"))

In [ ]:
#mostInfluentialValues(index)
    #retorna 3 dataframes, primero retorna grupos de indicadores que se correlaciona entre ellos en mas de 75%
    #segundo retorna el el promedio de la suma de cuadrados de cada indicador sobre los otros indicadores
    #ultimo retorna el mas influyente de un grupo y a los que influye
#Parameters:
    #index:getCorreatedVariables
        #data frame resultado de la funcion getCorreatedVariables donde tiene como columnas X Y Val
def mostInfluentialValues(index):
    copy_index = index.copy().set_index(["X","Y"])
    copy_index_list = copy_index.index.tolist()
    #mean_x = copy_index.groupby(by="X")["Val"].apply(lambda c: np.sqrt(c**2).mean()).sort_values(ascending=False)
    group_x = index.copy().groupby(by="X")['Y'].apply(list).to_dict()
    group_x = sorted(group_x.items(), key = lambda item : len(item[1]), reverse=False)
    grupos = []
    for key, val in group_x:
        grupo = list(np.concatenate([val,[key]]))
        grupo_aux = grupo.copy()
        for i in range(len(grupo)-1):
            for j in range (i+1,len(grupo)):
                if (grupo[i], grupo[j]) in copy_index_list or (grupo[j], grupo[i]) in copy_index_list:
                    pass
                else:
                    if grupo[j] in grupo_aux:
                        grupo_aux.remove(grupo[j])
        grupos.append(grupo_aux)
    i=0
    while i!=len(grupos)+1:
        for indicadores in grupos[i+1:]:
            flag = True
            for ind in grupos[i]:
                if ind not in indicadores:
                    flag=False
                    break
            if flag:
                grupos.pop(i)
                grupos= sorted(grupos, key=len)
                i =-1
                break
        i +=1

        
    ls_val_inf = []
    more_inf = {}
    for index, val in enumerate(grupos):
        grupo = val
        inf_val = []
        for i in range(len(grupo)):
            cont = 0
            for j in range (len(grupo)):
                if i!=j:
                    if (grupo[i], grupo[j]) in copy_index_list:
                        cont +=copy_index.loc[(grupo[i], grupo[j])]["Val"]**2
                    if (grupo[j], grupo[i]) in copy_index_list:
                        cont +=copy_index.loc[(grupo[j], grupo[i])]["Val"]**2
            inf_val.append(cont/(len(grupo)))
        ls_val_inf.append(inf_val)
        more_inf[grupos[index][inf_val.index(max(inf_val))]]=grupo
    return grupos, ls_val_inf, more_inf #, group_val

In [ ]:
#run_mostInf(sectors, inName)
    #retorna dos dataframes, donde el primero es todo las variables mas influyentes de todo los sectores de un pais y el segundo son
    #los indicadores independientes que no tiene relacion ninguna con otro indicador
#Parameters:
    #sector: Array[String]
        #array con los sectores pertinentes de un pais, cada sector esta ubicado dentroi de una pestaña de excel
    #inName: String
        #nombre del archivo excel donde se encuentra las matrices de correlacion de cada sector de un pais
def run_mostInf(sectors, inName):
    xlsx = excelFile(inName)
    dict_more_inf = {}
    free_ind = []
    max_length = 0
    all_ind = []
    print(sectors)
    for sector in sectors:
        print(sector)
        corr_var = pd.read_excel(xlsx, sheet_name=sector).set_index("index")
        corr_columns = list(corr_var.columns)
        corr_var = getCorreatedVariables(corr_var,0.75, True)
        grupos, _, more_inf = mostInfluentialValues(corr_var)
        for grupo in grupos:
            all_ind = np.concatenate((all_ind, grupo), axis=None)
        all_ind = np.unique(all_ind)
        for col in corr_columns:
            if col not in all_ind:
                free_ind += [col]    
        for key, val in more_inf.items():
            dict_more_inf[key]=val 
            if max_length<len(val):
                max_length = len(val)
    dict_all_inf_dataframe = {}         
    for key,val in dict_more_inf.items():
        end = [None] * (max_length - len(val))
        dict_all_inf_dataframe[key]=np.concatenate([val,end])
        
    return dict_all_inf_dataframe, free_ind

#sectors = ['Education','Environment']
#inf, free_ind = run_mostInf(sectors, "corr_multiple.xlsx")

In [ ]:
#finalIndicators(dataFrame, inf, freeInd)
    #Retorna los indicadores de estudio para la imputacion
#Parameters:
    #dataFrame: DataFrame
        #Resultado de la funcion outlier
    #inf:Dict
        #Indicadores mas influyentes, primer resultado de la funcion run_mostInf
    #free_ind: list[String]
        #Indicadores independientes, segundo resultado de la funcion run_mostInf
def finalIndicators(dataFrame, inf, freeInd):
    ind = np.concatenate([list(inf.keys()), freeInd])
    return dataFrame.loc[:,ind]

#data_frame = readFile("delIndicatorMayorUmbralZerosOrNan.xlsx", "BRA")
#data_frame = finalIndicators(data_frame, inf, free_ind)
#data_frame.head()

In [ ]:
#runPreprocess(wdi, series, paises, añosPais)
    #Realiza todo los pasos de preprocesamiento de los paises especificados, genera dos archivos cod_pais.xlsx y cor_codpais.xlsx
#Parameters:
    #wdi:DataFrame
        #Archivo excel WDI.xlsx
    #series:DataFrame
        #Archivo excel series WDI_CETS.csv or xlsx
    #paises:List[String]
        #lista de codigos de paises existentes en WDI
    #añosPais:Dict{key_code_pais:[añoInit,añoEnd]}
        #diccionario donde el key es el codigo d pais existente en WDI y la lista correspondiente a esa key
        #es el año inicial de analisis y el año final de analisis
def runPreprocess(wdi, series, paises, añosPais):
    if not os.path.exists(path_file):
        os.mkdir(path_file)
    if not os.path.exists(path_file+"/splitByColumn.xlsx"):
        splitByColumn(wdi, "Country Code")
        print("splitByColumn.xlsx file made  Successfully in {}".format(path_file))
    del wdi
    gc.collect()
    for pais in paises:
        print("Realizando el preprocesamiento para {}...".format(pais))
        with pd.ExcelWriter('datos_reales/{}.xlsx'.format(pais)) as writer:
            #corr.to_excel(writer, sheet_name=sheet, index=True)
            data=readFile('splitByColumn.xlsx', pais)
            miss_year = countMissingDataPerYear(data,añosPais[pais][0],añosPais[pais][1])
            miss_year.to_excel(writer, sheet_name="faltantes por año", index=True)
            del miss_year
            gc.collect()

            miss_ind = countMissingDataPerIndicator(data,añosPais[pais][0],añosPais[pais][1])
            miss_ind.to_excel(writer, sheet_name="faltantes por ind", index=True)
            del miss_ind
            gc.collect()

            data= dataFrame(data, añosPais[pais][0],añosPais[pais][1])


            menor_umbral=delIndicatorMayorUmbralZerosOrNan(data,25)
            menor_umbral.to_excel(writer, sheet_name="missing menor 25", index=True)

            menor_umbral_copy = menor_umbral.copy().reset_index()
            without_change = delIndicatorWithoutChanges(menor_umbral_copy,True)
            without_change.to_excel(writer, sheet_name="ind sin variacion", index=True)
            del menor_umbral_copy
            gc.collect()

            without_change = delIndicatorWithoutChanges(menor_umbral.reset_index(),False)

            data_no_atipico, data_atipicos = outlier(without_change)
            data_atipicos.to_excel(writer, sheet_name="ind eliminados en outlier", index=True)
            
            series_wdi = joinSeriesAndWdi(series,data_no_atipico.reset_index())
            del data_no_atipico
            gc.collect()

            grupos = splitByColumn(series_wdi.reset_index(), by="Topic", frame=True)
            sectores = list(grupos.groups.keys())
            runCorrelation(grupos, out = "corr_{}.xlsx".format(pais))
            
            numberIndicatorsBySector(grupos).to_excel(writer, sheet_name="Cant Indicadores", index=True)
            del grupos
            gc.collect()
            
            inf, free_ind = run_mostInf(sectores, "corr_{}.xlsx".format(pais))
            dataframe_inf = pd.DataFrame(inf)
            dataframe_inf.columns = dataframe_inf.columns.str.replace(".",".")
            dataframe_inf.to_excel(writer, sheet_name="influyentes", index=False)
            del dataframe_inf
            gc.collect()
            
            pd.DataFrame(free_ind).to_excel(writer, sheet_name="sin correlacion", index=True)
            
            data_no_atipico, _ = outlier(menor_umbral)
            final_data_frame = finalIndicators(data_no_atipico, inf, free_ind)
            final_data_frame.to_excel(writer, sheet_name="influyentes y sin correlacion", index=True)
            del final_data_frame
            gc.collect()
    if os.path.exists(path_file+"/splitByColumn.xlsx"):
          os.remove(path_file+"/splitByColumn.xlsx")
    else:
          pass
    print("Se realizo el preprocesamiento con exito!!!")

#paises = ['ARG','BOL', 'BRA', 'CHL', 'COL', 'ECU','GUY', 'PER', 'PRY','SUR','TTO', 'URY', 'VEN']
#paises = ['BRA']
#añosPais = {'ARG':[1991,2016],'BOL':[1990,2017], 'BRA':[1990,2017], 'CHL':[2002,2016],
#             'COL':[1990,2017], 'ECU':[1993,2017],'GUY':[2000,2015], 'PER':[1990,2017], 
#             'PRY':[1995,2017],'SUR':[None,None],'TTO':[None,None], 'URY':[2000,2016], 'VEN':[1991,2014]}
#wdi= readFile("wdi_0.1.xlsx","Data")
#series = readFile("seriesCETS.csv")
#runPreprocess(wdi, series, paises, añosPais)

# 3. Imputacion de indicadores en R

# 4. Normalizacion

In [ ]:
#normalizeData(dataFrame, transpose=False)
    #Normaliza un dataframe estableciendo la media en 0 y la desviacion estandar en 1
#Parameters:
    #dataFrame:DataFrame
        #dataframe a normalizacion, donde las columnas son los indicadores
    #transpose:
        #Si es true transpone el resultado, caso contrario retorna la misma estructura ingresada
def normalizeData(dataFrame, transpose=False):
    #std_scale = preprocessing.StandardScaler().fit(_dataFrame)
    #data_norm = std_scale.transform(_dataFrame)
    data_norm = preprocessing.StandardScaler().fit_transform(dataFrame)
    
    data_norm_col = pd.DataFrame(data_norm, index=dataFrame.index, columns=dataFrame.columns) 
    dataFrame.update(data_norm_col)
    if transpose:
        return dataFrame.T
    return dataFrame

### Normalize multidata
https://github.com/EL-BID/Agregador-de-indicadores/blob/master/docs/Normalizacion.md

In [ ]:
#para salvar la normalizacion para VEN año=2017, se repite el ultimo valor que es el año 2016(La variable EG.ELC.ACCS.ZS
#esta correlacionada con el año, por lo que no se puede imputar)
def NormalizeMultiData(_file, _paises):
    xlsx = excelFile(_file)
    dataFrame = []
    suma = 0
    dev = 0
    #calcula la media
    for pais in _paises:
        data = pd.read_excel(xlsx, sheet_name="{0}".format(pais)).set_index('year')
        dataFrame.append(data)
        suma+=data.values
    media = suma/len(_paises)
    #calcula la desviacion estanda poblacional
    for frame in dataFrame:
        dev+=np.power(frame.values-media,2)
    dev = np.sqrt(dev/len(_paises))
    #guarda los datos normalizados
    with pd.ExcelWriter('normalizeMultiData.xlsx') as writer:
        for i,frame in enumerate(dataFrame):
            frame.iloc[:,:] = ((frame.values-media)/dev)*-1
            frame.to_excel(writer, sheet_name=str(_paises[i]), index=True)
'''
paises = ['ARG','BOL', 'BRA', 'CHL', 'COL', 'ECU','GUY', 'PER', 'PRY','SUR','TTO', 'URY', 'VEN']
NormalizeMultiData('imputacion.xlsx', paises)
'''

# 5. Modelo

## DMMTS
Dynamic multi time series aplicando distancia de mahalanobis en multi time series

In [ ]:
#realiza la covarianza de dos Multi time series X, Y
#Observacion: Las columnas deben de ser las variables
def pooledCov(X,Y):
    '''if X.ndim == 1:
        X = np.reshape(np.array(X),(1,-1))
    if Y.ndim == 1:
        Y = np.reshape(np.array(Y),(1,-1))'''
    cvX = np.dot(X,X.T)/X.shape[1]
    cvY = np.dot(Y,Y.T)/Y.shape[1]
    #cvY = np.cov(Y)        
    x1 = X.shape[1]
    x2 = Y.shape[1]
    return (x1/(x1+x2))*cvX + (x2/(x1+x2))*cvY

In [ ]:
#Mahalanobis distance para dos series de tiempo
#Observacion: Las columnas deben de ser las variables
def mahalanobis(x=None, y=None):
    #centramos los datos en su media aritmetica
    a = x-np.mean(x,axis=1,keepdims=True)
    b = y-np.mean(y,axis=1, keepdims=True)
    #aplicamos la covarianza
    inv_covmat = np.linalg.inv(pooledCov(a,b))
    mean= np.mean(x, axis=1)-np.mean(y, axis=1)
    shape = mean.shape[0]
    mean = mean.reshape(-1,shape)
    return np.sqrt(np.squeeze(np.dot(mean,np.dot(inv_covmat,mean.T))))

In [ ]:
#Busca el camino optimo de la matriz Dynamic time warping (W)
def findPath(W):
    i, j = W.shape
    i=i-1
    j=j-1
    path = []
    while (i, j) != (0, 0):
        if i < 0:
            i=0
        if j < 0:
            j=0
        path.append((i, j))
        i, j = min((i - 1, j), (i, j - 1), (i - 1, j - 1), key = lambda x: W[x[0], x[1]]) 
    path.append((0,0))
    return W[-1, -1], path

In [ ]:
#Determina la similitud entre dos multi time series, utilizando Dinamic time warping y la distancia de mahalanobis
#X,Y (matrices): Grupos con multiples series, las columnas son la serie y las filas son las variables
def DMMTS(X,Y,window=0):
    W  = sys.maxsize * np.ones((X.shape[1],Y.shape[1]))
    if X.shape[0]==1 and Y.shape[0]==1:
        W[0,0] = np.absolute(X[:,0].reshape(1,-1)-Y[:,0].reshape(1,-1))
        x_aux = [np.absolute(X[:,0].reshape(1,-1)-Y[:,i].reshape(1,-1))+W[0,i-1] for i in range(1,Y.shape[1])]
        y_aux = [np.absolute(X[:,1].reshape(1,-1)-Y[:,i].reshape(1,-1))+W[i-1,0]  for i in range(1,X.shape[1])]
        W[0,1:]=x_aux
        W[1:,0]=y_aux
        for i in range(1,X.shape[1]):
            for j in range(max(1, i - window), min(Y.shape[1], i + window)):
                W[i,j] = np.absolute(np.array(X[:,i]).reshape(1,-1)-np.array(Y[:,j]).reshape(1,-1)) + np.amin([W[i-1,j],W[i-1,j-1],W[i,j-1]])
    else:
        W[0,0] = mahalanobis(X[:,0].reshape(1,-1),Y[:,0].reshape(1,-1))
        x_aux = [mahalanobis(X[:,0].reshape(1,-1),Y[:,i].reshape(1,-1))+W[0,i-1] for i in range(1,Y.shape[1])]
        y_aux = [mahalanobis(X[:,1].reshape(1,-1),Y[:,i].reshape(1,-1))+W[i-1,0]  for i in range(1,X.shape[1])]
        W[0,1:]=x_aux
        W[1:,0]=y_aux
        for i in range(1,X.shape[1]):
            #for j in range(1, Y.shape[1]):
            for j in range(max(1, i - window), min(Y.shape[1], i + window)):
                W[i,j] = mahalanobis(np.array(X[:,i]).reshape(1,-1),np.array(Y[:,j]).reshape(1,-1)) + np.amin([W[i-1,j],W[i-1,j-1],W[i,j-1]])
                #W[i,j] = np.array(X[:,i]).reshape(1,-1)-np.array(Y[:,j]).reshape(1,-1) + np.amin([W[i-1,j],W[i-1,j-1],W[i,j-1]])
    costo, path = findPath(W)       
    return W, costo, path

In [ ]:
#Grafica la ruta optima
#W: matriz Dinamic time warping , path: coordenadas de la ruta optima(x,y)
def plotPath(W, path):
    x,y = zip(*path)
    plt.figure(figsize=(10,10))
    plt.imshow(W, origin='lower', interpolation='nearest')
    plt.colorbar()
    plt.scatter(x,y,color='r')
    plt.plot(x, y, color="r")
    plt.show()

In [ ]:
#Grafica como se cogio las distancias entre indicadores
#X,Y: Grupo con multi time series, las filas son las variables y las columnas las series
#path: coordenadas de la ruta optima(x,y)
def plotDistance(X,Y, path):
    offset = 5
    plt.figure(figsize=(10,10))
    plt.xlim([-1, max(len(X.T), len(Y.T)) + 1])
    plt.plot(X.T)
    plt.plot(Y.T + offset)
    for (x1, x2) in path:
        plt.plot([x1, x2], [X.T[x1], Y.T[x2] + offset])
    plt.show()

## Ejecucion del modelo

### Clustering


In [ ]:
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, ward,average
from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import pdist
from scipy.spatial.distance import mahalanobis as mh
from collections import defaultdict
from matplotlib.colors import rgb2hex, colorConverter
from scipy.cluster import hierarchy
#Probar si funciona
#import pdfkit
import statsmodels.api as sm
import matplotlib as mpl

In [ ]:
#Clusters(dict):#convierte cada etiqueta "color hexadecimal" en su respectivo color
#Parameters:
    #pathWkhtml:String
        #ruta del programa wkhtmltopdf.exe, si es None solo muestra la tabla y no realiza el proceso de guardado
        #OJO: instalar wkhtmltopdf
    #OutName:String
        #Nombre del archivo pdf de salida, almacena los clusters encontrados del dendrograma en formato de tabla 
class Clusters(dict):
    def _repr_html_(self, pathWkhtml = "C:\src\wkhtmltopdf\bin\wkhtmltopdf.exe", outName="out.html"):
        html = '<table style="border:"1"; font-family:"Arial"; font-size:"16"; ">'
        html += '<tr><th style="border: 1px solid black; text-align: center">Cluster</th>'
        html += '<th style="border: 1px solid black; text-align: center">Indicator</th>'
        html += '<th style="border: 1px solid black; text-align: center">Total</th></tr>'
        for i, c in enumerate(self):
            hx = rgb2hex(colorConverter.to_rgb(c))
            html += '<tr style="0">' \
            '<td style="background-color: {0}; text-align: center' \
                       'border: 0;">' \
            '<code style="background-color: {0};">'.format(hx)
            html += "Cluster {0}".format(i+1) + '</code></td>'
            html += '<td style="border: 1px solid black; text-align: center"><code>'
            html += repr(self[c]) + '</code></td>'
            html +='<td style="border: 1px solid black; text-align: center"><code>'
            html += str(len(self[c]))+'</code></td></tr>'
        html += '</table>'
        with open(outName, "w") as file:
            file.write(html)
            #si pdfkit funciona, activar las siguientes lineas
        ''' if pathWkhtml is not None:
            path_wkthmltopdf = r'{}'.format(pathWkhtml)
            config = pdfkit.configuration(wkhtmltopdf=path_wkthmltopdf)
            pdfkit.from_string(html, os.path.join(path_file,outName) , configuration=config)
        else:
            print("Instale el programa wkhtmltopdf y estableza la ruta en el modulo Clusters para guardarlo: Ejemplo {}'.format(pathWkhtml)")
        '''
        return html

In [ ]:
#retorna las etiquetas de cada cluster en funcion de la distancia
def get_cluster_classes(den, max_d, label='ivl'):
    cluster_idxs = defaultdict(list)
    for c, pi, d in zip(den['color_list'], den['icoord'], den['dcoord']):
        for leg in pi[1:3]:
            i = (leg - 5.0) / 10.0
            if abs(i - int(i)) < 1e-5:
                y = d[1]
                if y <= max_d:
                    if i not in cluster_idxs[c]:
                        cluster_idxs[c].append(int(i))
    cluster_classes = Clusters()
    for c, l in cluster_idxs.items():
        i_l = [re.sub("&", "and", den[label][i]) for i in l]
        cluster_classes[c] = i_l

    return cluster_classes

In [ ]:
#las graficas anteriores son erroneas por que no se considera la distancia correcta
def Elbow(dendrogram, name="dist.png", size=(15,5), posicion=0):
    dist =[]
    for d in dendrogram['dcoord']:
        dist.append(d[1])
    dist = sorted(dist,key=float)
    xaxis= np.arange(0,len(dist))
    plt.figure(figsize=size)
    hfont = {'fontname':'Arial'}
    font_size = 16
    plt.title("Optimal Distance", fontsize=15, **hfont)
    plt.xlabel("Step", fontsize=font_size, **hfont)
    plt.ylabel("Distance", fontsize=font_size, **hfont)
    plt.annotate("Optimal Distance %.6g" % dist[posicion], (posicion, dist[posicion]), xytext=(-25, 20),
                             textcoords='offset points',
                             va='top', ha='center')
    plt.plot(xaxis,dist, linestyle='dashed', linewidth=2, markersize=12, label="Distance")
    plt.axvline(color='red',x=posicion, linestyle='--')
    plt.grid(True)
    plt.legend(bbox_to_anchor=(0.01, -0.01, 1, 1), loc='upper left', ncol=1, borderaxespad=0)
    #plt.title("Elbow")
    plt.savefig(os.path.join(path_file,name), bbox_inches = 'tight',pad_inches = 0)
    return dist
#Elegir el paso correcto hasta el punto de quiebre
#EJEMPLO: Para BOL el paso obtenido era 0.1... pero de acuerdo al analisis se decide colocar en el punto de quiebre "260"

In [ ]:
#realiza el dendrograma con las distancias obteninas
#el metodo de agrupacion utilizada es ward
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', 0)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)
    size = kwargs.pop('size', (15,10))
    plt.figure(figsize=size)
    hfont = {'fontname':'Arial'}
    font_size = 16
    no_plot_table = kwargs.pop('no_plot_table', None)
    ddata = dendrogram(*args, **kwargs)
    clusters = None
    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram With Ward Method', fontsize=font_size, **hfont)
        plt.xlabel('Sample Index or (cluster size)', fontsize=font_size, **hfont)
        plt.ylabel('Distance', fontsize=font_size, **hfont)
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
        plt.savefig(os.path.join(path_file,"dendrograma.png"), bbox_inches = 'tight',pad_inches = 0)
        plt.show()
    if not no_plot_table: 
        clusters = get_cluster_classes(ddata, max_d, label='ivl')
    return ddata, clusters

### Indicadores

In [ ]:
#procesa un dataframe mediante DMMTS
#_dataFrame: dataframe, donde tiene como columnas los indicadores, y filas las series temporales
#skeep: numero de indicadores a agrupar
#windows:tamaño de la ventana para DMMTS
#Observacion:Verificar que no exista # N/A
def DataFrameForClustering(_dataFrame, skeep=2, windows=10):
    columnas =_dataFrame.shape[1]
    costos = []
    if columnas%skeep!=0:
        raise Exception('El numero de columnas {} debe ser multiplo de las agrupaciones {}'.format(columnas, skeep))
    else:
        for i in range(0,columnas,skeep):
            p1 = _dataFrame.iloc[:,i:i+skeep]
            for j in range(i+skeep,columnas,skeep):
                p2 = _dataFrame.iloc[:,j:j+skeep]
                _,costo,_= DMMTS(p1.values.T,p2.values.T,windows)
                costos.append(costo)
    return costos
# sentencias para iniciar la normalizacion
file = readFile('ARG_SECTORES.xlsx','economic policy & debt').set_index('year')
file = normalizeData(file)
#inicia el calculo de las distancias por indicador de pais
#DataFrameForClustering(file,1) el uno inidica el rango de compracion, en este caso es de uno a uno
distancias_similitud = DataFrameForClustering(file,1)

In [ ]:
#VARIAR numero_cluster
numero_cluster = 13
cmap = plt.cm.rainbow(np.linspace(0, 1, numero_cluster)) #Variar de acuerdo al numero de clusters que se obtiene
hierarchy.set_link_color_palette([mpl.colors.rgb2hex(rgb[:3]) for rgb in cmap])
#METODO WARD
linked_array = ward(np.array(distancias_similitud))
dn, clusters = fancy_dendrogram(
    linked_array,
    truncate_mode='lastp',
    p=400,
    #activar para reporte de indicadores por pais
    labels=[i for i in file.columns],#cambiar el arreglo para cambiar las etiquetas del cluster
    no_plot=False, #plot dendrogram
    no_plot_table =False,#grafica la tabla de clusters
    orientation='top',
    leaf_rotation=90.,
    leaf_font_size=9.,
    show_contracted=True,
    annotate_above=1,
    size=(25,10), #tamaño (width, height) tamaño de configuracion para ind=75,10  sec=15,10, det_sec=25,10
    #Activar despues de ejecutar el metodo Elbow con el Step configurado, ejemplo BOL=260
    max_d=dist[140]+0.1 #distancia maxima para determinar el numero de cluster 
)
#Observacion: Los archivos generados se remplaza en cada ejecucion
#funcion elbow
dist=Elbow(dn, size=(15,5), posicion=140)
#Instalar el programa del modulo Clusters(dict) y configurar el parametro del modulo con la ruta donde se ha instalado
clusters

#### plot ind
media de indicadores, mini graficos de tendencias, tabla resumen de clusters por año y clusters. 

In [ ]:
#plotAllClustersIndicators(dataFrame, clusters, cluster_title): genera media de indicadores, mini graficos de tendencias, tabla resumen de clusters por año y clusters.
#Parameters:
    #dataFrame:DataFrame
        #dataframe a analizar
    #clusters: Array[String]
        #array de indicadores
    #cluster_title: String
        #Titulo del grafico
def plotAllClustersIndicators(dataFrame, clusters, cluster_title):
    font_size = 16
    hfont = {'fontname':'Arial'}
    data_frame = dataFrame
    plt.figure(figsize=(15,5))
    plt.subplot()
    means = {}
    colores = []
    plt.xlabel("Year", fontsize=font_size, **hfont)
    plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
    i = 1
    for key, val in clusters.items():
        mean_cluster = data_frame.loc[:,val].mean(axis=1)
        name = "Cluster "+ str(i)
        means[name] = mean_cluster
        p = plt.plot(mean_cluster,label=name, marker='o', linewidth=1.5)
        colores.append(p[0].get_color())
        i +=1
    plt.legend(bbox_to_anchor=(0.0, 0.39, 1, 1), loc='upper center', ncol=7 , mode="expand", borderaxespad=0)
    plt.grid(True)   
    plt.title(str(cluster_title).title(), fontsize=font_size, **hfont)
    plt.savefig(os.path.join(path_file,"media.png"), bbox_inches = 'tight',pad_inches = 0)
    #Grafica subgraficos del cluster 
    x0,x1,y0,y1 = plt.axis() #obtiene los puntos iniciales y finales del eje x e y
    i = 1
    plt.figure(figsize=(15,20))
    for key, val in means.items():
        x1=plt.subplot(7, 4, i)
        plt.ylim(y0, y1)
        #plt.title(key.title(),fontsize=12, **hfont)
        ciclo, tend = sm.tsa.filters.hpfilter(val, lamb=6.25)#6.25 es para años y 1000 u otros valores para dias, consultar wdi.txt
        plt.plot(val, label=key, marker='', linewidth=1.7, color=colores[i-1])#cluster
        plt.scatter(list(tend.index),tend,label="Tendencia", marker='.', linewidth=0.3, color="gray")#tendencia
        plt.setp(x1.get_xticklabels(), visible=True)
        plt.legend(bbox_to_anchor=(0, 0, 1, 1), fontsize=8)#coloca legend inteligentemente sin tapar la grafica
        plt.grid(True)
        i +=1

    plt.subplots_adjust(top=1.0, bottom=0.08, left=0.10, right=0.95, hspace=0.30, wspace=0.15)
    plt.savefig(os.path.join(path_file,"tendencias.png"), bbox_inches = 'tight',pad_inches = 0)
    plt.show()
    return means
    #plt.savefig('Representativo.png',bbox_inches = 'tight',pad_inches = 0)
dataFrame = readFile("ARG_SECTORES.xlsx","economic policy & debt")
dataFrame = normalizeData(dataFrame.set_index("year"))
cluster_title = "Economic policy and debt from Argentina"
mean = plotAllClustersIndicators(dataFrame, clusters, cluster_title)
saveFile(pd.DataFrame(mean).T.describe().T, "describe_cluster_anio.xlsx")
saveFile(pd.DataFrame(mean).describe().T, "describe_cluster.xlsx")

Reporte de sectores en un archivo excel, cuando join es True retorna en unico archivo excel los sectores. <br>
Observacion: Reporte unicamente para indicadores por pais.

In [ ]:
#unir(x)
    #funcion usada en agg de clustersDescripcion, antepone una coma a un string 
#Parameters:
    #x:String
        #String a anteponer la coma
def unir(x):
    return ", ".join(x)

def totalInd(inf, indInf):
    aux = indInf.copy()
    contador = 0
    for ind in indInf:
        if ind not in inf.columns:
            aux.remove(ind)
            contador +=1
    if len(aux)==0:
        return contador
    inf = np.unique(inf.loc[:,aux].values.tolist()).shape[0]-1
    return inf+contador

#clustersDescripcion(series, cluster, by="Topic")
    #Recupera los sectores de los indicadores que forman un cluster, los sectores se obtienen de series
#Parameters:
    #series: DataFrame
        #dataframe del archivo seriesCets.csv
    #cluster:List(String)
        #Lista de todo los indicadores que forman un cluster
    #total:DataFrame
        #dataframe que contiene la cantidad de indicadores influyentes de un sector de un determinado pais[index(topico), column=Nro]
    #by:String
        #Columna de seriesCets mediante el cual agrupar

def clustersDescripcion(series, cluster,total=None, inf=None, by="Topic"):
    series = series.set_index("Series Code").T
    if total is None and inf is None:
        return series.loc[[by],cluster].T.reset_index().groupby(by=by).agg(Cod_ind=('Series Code',unir),
                                                                       Nro=("Series Code",'count'))
    else:
        mezcla = pd.merge(series.loc[[by],cluster].T.reset_index().groupby(by=by).agg(Cod_ind=('Series Code',unir),
                                                                      Nro=("Series Code",'count')), total, on=by, how="inner")
        mezcla["%"] = round((mezcla["Nro_x"]*100)/mezcla["Nro_y"],2)
        mezcla.columns = ["Indicadores influyentes e independientes (Inf)","# A","# A por sector","% B"]
        #A:total de influyentes mas independientes
        #B:el porcentaje de A sobre #A por sector
        total_inds_por_grupo = []
        for inds in mezcla.loc[:,"Indicadores influyentes e independientes (Inf)"]:
            l = list(map(lambda x:x.strip(), inds.split(",")))
            total_inds_por_grupo.append(totalInd(inf,l))
        mezcla["C"] = total_inds_por_grupo ## numero total de influyentes, influidos e independientes
        mezcla.sort_index(inplace=True)
        return mezcla
#nro = 1
#cluster = clusters[list(clusters.keys())[nro-1]]
#series = readFile("seriesCets.csv")
#series = clustersDescripcion(series, cluster)

#runClustersDescripcion(series, clusters, outName)
    #Guarda el reporte de todo los clusters de un pais
#Parameters:
    #series:DataFrame
        #dataframe del archivo seriesCets.csv
    #inf:DataFrame
        #dataframe del archivo influyentes de un pais especifico a ser analizado
    #clusters:dict
        #diccionario con los clusters de un pais, se obtiene del reporte del dendrograma
    #outName:String
        #nombre del reporte resultante
    #join: Boolean
        #si es true guarda el reporte de cluster por sector agrupado en un excel, caso contrario cada sector en pestañas diferentes
def runClustersDescripcion(series, inf, clusters, outName, join):
    with pd.ExcelWriter('datos_reales/{}.xlsx'.format(outName)) as writer:
        values = []
        for key, val in clusters.items():
            values +=val
        total=clustersDescripcion(series, values).iloc[:,1]
        if join:
            mini_col = ["A","% B","C"]
            tuple_columns = [(sector,m) for sector in total.index for m in mini_col]
            cols = pd.MultiIndex.from_tuples(tuple_columns)
            join_data_frame = pd.DataFrame(columns=cols, index=np.arange(1,len(clusters)+1))
            i = 1
            for key, val in clusters.items():
                resumen = clustersDescripcion(series, list(val), total, inf).iloc[:,[1,3,4]].T
                for column in resumen.columns:
                    join_data_frame.loc[i,column]=list(resumen[column])
                i +=1
            totales = join_data_frame.sum(axis=0)
            join_data_frame.loc["Total"]= totales
            join_data_frame.index.name = "Clusters"
            join_data_frame.to_excel(writer, sheet_name="{}".format(outName[:3]), index=True)
        else:
            i = 1
            for key, val in clusters.items():
                clustersDescripcion(series, list(val), total, inf).to_excel(writer, sheet_name="Cluster{}".format(str(i)), index=True)
                i +=1
                
series = readFile("seriesCets.csv")
influyentes = readFile("ARG.xlsx","influyentes")
#outName: cambiar de acuerdo al pais analizado
runClustersDescripcion(series, influyentes, clusters, outName="ARG_clusters_join", join=True)
runClustersDescripcion(series, influyentes, clusters, outName="ARG_clusters", join=False)

Grafica un cluster especifico de indicadores 

In [ ]:
#Funcion: grafica un cluster del reporte indicador por pais
#urlFile: archivo excel de un pais separado por sectores
#sheet_name: nombre de hoja del archivo excel
#_namesInClusters: array de string con los nombres de los indicadores
#title: titulo del grafico
#colEtiquetas: numero de columnas de la leyenda
#normalizacion: si es True normaliza los datos
#opcion: si es 0 grafica la media del cluster, caso contrario grafica los indicadores del cluster
def plotClustersIndicators(urlFile, sheet_name, _namesInClusters, title, colEtiquetas, normalizacion=True, opcion = 0):
    xlsx = excelFile(urlFile)
    font_size = 16
    hfont = {'fontname':'Arial'}
    if normalizacion:
        indicadores=normalizeData(pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters])
    else:
        indicadores=pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters]
    vmin = math.floor(indicadores.min().min())
    vmax = math.ceil(indicadores.max().max())
    plt.figure(figsize=(14,5))
    plt.subplot()
    plt.ylim(vmin, vmax)
    if opcion ==0:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
        for i in indicadores:
            plt.plot(indicadores.loc[:,i],label=str(i), marker='', linewidth=1.5)
        col = math.ceil(len(indicadores)/colEtiquetas)
        plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(indicadores)/col), mode="expand", borderaxespad=0)
    if opcion ==1:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Average Normalized Value", fontsize=font_size, **hfont)
        plt.plot(indicadores.mean(axis=1),label="media", marker='', linewidth=1.5)   
        plt.legend(bbox_to_anchor=(0.01, -0.01, 1, 1), loc='upper left', ncol=1, borderaxespad=0)
    plt.grid(True)   
    plt.title(("Cluster "+str(title)).title(), fontsize=font_size, **hfont)
    plt.show()
    #plt.savefig('Representativo.png',bbox_inches = 'tight',pad_inches = 0)

    #return indicadores
urlFile = 'ARG_SECTORES.xlsx'
sheet_name = 'economic policy & debt'
cluster = 12
_namesInClusters = clusters[list(clusters.keys())[cluster-1]]
colEtiquetas = 5
normalizacion = True
plotClustersIndicators(urlFile, sheet_name, _namesInClusters, cluster, colEtiquetas, normalizacion, opcion = 0)
plotClustersIndicators(urlFile, sheet_name, _namesInClusters, cluster, colEtiquetas, normalizacion, opcion = 1)

### Sectores

In [ ]:
#splitByTopicNormalizeData(series, normalize, outName)
    #Une los archivos serieCets y el resultado de normalizar, luego se agrupa por grupos(Topic) y se guarda en un archivo excel 
#Parameters:
    #series:DataFrame
        #dataframe del archivo seriesCets.csv
    #dataFrame:DataFrame
        #data frame a analizar
    #outName:String
        #Nombre del archivo resultado
def splitByTopicNormalizeData(series, dataFrame, outName):
    union = joinSeriesAndWdi(series, dataFrame)
    with pd.ExcelWriter('datos_reales/{}.xlsx'.format(outName)) as writer:
        for key, group in union.groupby(by="Topic"):
            group = group.iloc[:,5:].T
            group.index.name = "year"
            group.to_excel(writer, sheet_name=str(key).lower(), index=True)
    print("Succesfully complete!!!!")
wdi = readFile("BOL.xlsx","imp")
wdi = normalizeData(wdi)
series = readFile("seriesCets.csv")
splitByTopicNormalizeData(series, wdi, outName="BOL_SECTORES")

In [ ]:
#procesa un excel mediante DMMTS, solo para sectores por pais
#sectores: Lista de sheet en el archivo excel 
def ExcelDataForClustering(urlFile, sectores):
    costos = []
    matrix = np.zeros(())
    xlsx = excelFile(urlFile)
    for i,data in enumerate(sectores):
        p1 = pd.read_excel(xlsx, sheet_name="{0}".format(data)).set_index("year")
        for j in range(i+1, len(sectores)):
            p2 = pd.read_excel(xlsx, sheet_name="{0}".format(sectores[j])).set_index("year")
            _,costo,_= DMMTS(p1.values.T,p2.values.T,5)
            costos.append(costo)
    return costos


#retorna los sectores con datos
#urlFile: ingresar nombre del archivo excel estudiado dividido por sectores del modulo anterior (splitByTopicNormalizeData)
def buscarSectores(urlFile):
    sectores = []
    xlsx = excelFile(urlFile)
    for data in xlsx.sheet_names:
        if len(pd.read_excel(xlsx, sheet_name="{0}".format(data)).columns[1:])>0:
            sectores.append(data)
    return sectores

urFile = 'BOL_SECTORES.xlsx'
sectores = buscarSectores(urFile)
distancia_sectores = ExcelDataForClustering(urFile,sectores)

In [ ]:
#VARIAR numero_cluster
numero_cluster = 2
cmap = plt.cm.rainbow(np.linspace(0, 1, numero_cluster)) #Variar de acuerdo al numero de clusters que se obtiene
hierarchy.set_link_color_palette([mpl.colors.rgb2hex(rgb[:3]) for rgb in cmap])
linked_array = ward(np.array(distancia_sectores))
#z = hierarchy.cut_tree(linked_array, n_clusters=12)
dn, clusters = fancy_dendrogram(
    linked_array,
    truncate_mode='lastp',
    p=400,
    labels=[re.sub("&","and", i).title() for i in sectores],#cambiar el arreglo para cambiar las etiquetas del cluster
    no_plot=False, #plot dendrogram
    no_plot_table =False,#grafica la tabla de clusters
    orientation='top',
    leaf_rotation=90.,
    leaf_font_size=9.,
    show_contracted=True,
    annotate_above=1,
    size=(15,10), #tamaño (width, height)
    #Activar despues de ejecutar el metodo Elbow con el Step configurado, ejemplo BOL=260
    max_d=dist[6]+0.1 #distancia maxima para determinar el numero de cluster 
)
#Observacion: Los archivos generados se remplaza en cada ejecucion
#funcion elbow
dist=Elbow(dn, size=(15,5), posicion=6)
#Instalar el programa del modulo Clusters(dict) y configurar el parametro del modulo con la ruta donde se ha instalado
clusters

#### plot sectores

In [ ]:
#Funcion: grafica todo los indicadores pertenecientes a los sectores de un cluster 
#urlFile: archivo excel de un pais separado por sectores
#_namesInClusters: array de string con los nombres de los sectores a graficar
#colEtiquetas: las etiquetas de cada serie en cuantas columnas se desea agrupar
#opcion: 0:grafica el cluster, 1: grafica la media del cluster
def plotClustersSector(urlFile, _namesInClusters, colEtiquetas, opcion = 0):
    xlsx = excelFile(urlFile)
    sectores = pd.read_excel(xlsx, sheet_name="{0}".format(_namesInClusters[0])).set_index("year")
    for data in range(1,len(_namesInClusters)):
        aux=pd.read_excel(xlsx, sheet_name="{0}".format(_namesInClusters[data])).set_index("year")
        sectores = pd.concat([sectores,aux],axis=1)
    plt.figure(figsize=(14,5))
    vmin = math.floor(sectores.min().min())
    vmax = math.ceil(sectores.max().max())
    hfont = {'fontname':'Arial'}
    font_size = 16
    if opcion == 0:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
        for i in sectores:
            plt.plot(sectores.loc[:,i],label=str(i), marker='', linewidth=1.5)
        col = math.ceil(len(sectores)/colEtiquetas)
        plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(sectores)/col), mode="expand", borderaxespad=0)
    if opcion == 1:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Average Normalized Value", fontsize=font_size, **hfont)
        vmin = math.floor(sectores.mean(axis=1).min().min())
        vmax = math.ceil(sectores.mean(axis=1).max().max())
        plt.plot(sectores.mean(axis=1),label="media", marker='', linewidth=1.5)   
        plt.legend(bbox_to_anchor=(0.01, -0.01, 1, 1), loc='upper left', ncol=1, borderaxespad=0)
    #plt.plot(sectores.mean(axis=1),label="media", marker='', linewidth=2.5) 
    plt.ylim(vmin, vmax)
    plt.grid(True)
    plt.title(re.sub("&", "and", ", ".join(_namesInClusters)).title() + " - Cluster", fontsize=font_size, **hfont)
    plt.plot()
    #plt.savefig('Representativo.png',bbox_inches = 'tight',pad_inches = 0)

    #return sectores

In [ ]:
#Funcion: grafica todo los indicadores pertenecientes a un solo sector del mismo cluster anterior
#urlFile: archivo excel de un pais separado por sectores
#_namesInClusters: array de string con los nombres de los sectores a graficar
#colEtiquetas: las etiquetas de cada serie en cuantas columnas se desea agrupar
#opcion: 0:grafica el sector, 1: grafica la media del sector
#ymin = rango minimo en el eje Y para graficar
#ymax = rango maximo en el eje Y para graficar
def plotClustersSector1(urlFile, _namesInClusters, colEtiquetas, opcion = 0, ymin=-2, ymax=2):
    xlsx = excelFile(urlFile)
    font_size = 16
    hfont = {'fontname':'Arial'}
    for data in range(0,len(_namesInClusters)):
        sectores=pd.read_excel(xlsx, sheet_name="{0}".format(_namesInClusters[data])).set_index("year")
        plt.figure(figsize=(14,5))
        plt.subplot()
        plt.ylim(ymin, ymax)
        if opcion ==0:
            plt.xlabel("Year", fontsize=font_size, **hfont)
            plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
            for i in sectores:
                plt.plot(sectores.loc[:,i],label=str(i), marker='', linewidth=1.5)
            col = math.ceil(len(sectores)/colEtiquetas)
            plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(sectores)/col), mode="expand", borderaxespad=0)
        if opcion ==1:
            plt.xlabel("Year", fontsize=font_size, **hfont)
            plt.ylabel("Average Normalized Value", fontsize=font_size, **hfont)
            plt.plot(sectores.mean(axis=1),label="media", marker='', linewidth=1.5)   
            plt.legend(bbox_to_anchor=(0.01, -0.01, 1, 1), loc='upper left', ncol=1, borderaxespad=0)
        plt.grid(True)   
        plt.title(re.sub("sector", "", re.sub("&", "and", _namesInClusters[data])).title() + " Sector", fontsize=font_size, **hfont)
    plt.show()
    #plt.savefig('Representativo.png',bbox_inches = 'tight',pad_inches = 0)

    #return sectores

In [ ]:
urFile = 'BOL_SECTORES.xlsx'
cluster = 1
_namesInClusters = [re.sub("and", "&",i.lower()) for i in clusters[list(clusters.keys())[cluster-1]]]
#_namesInClusters = [re.sub("and", "&",i.lower()) for i in ['health']]
colEtiquetas = 12
data = plotClustersSector(urFile,_namesInClusters,colEtiquetas, opcion =0)
#ymin, ymax: parametros de escala
data1 = plotClustersSector1(urFile,_namesInClusters,colEtiquetas, opcion=0, ymin=-3, ymax=3)

### Descomposicion de una serie de indicadores
Explicacion de la ejecucion de la variable <strong>_namesInClusters</strong> usando un cluster<br>
1) Obtener los keys del diccionario<br>
 $keys=list(clusters.keys())$<br>
2) Seleccionamos una posicion de la lista de claves obtenidas en el paso 1 <br>
 $key = keys[1]$<br>
3) recuperamos los valores correspondientes a la clave del paso 2<br>
$clusters[key]$

In [ ]:
def plotRollingAverage(urlFile, sheet_name, _namesInClusters, title, colEtiquetas, normalizacion=True, opcion = 0, freq=5):
    xlsx = excelFile(urlFile)
    font_size = 16
    hfont = {'fontname':'Arial'}
    if normalizacion:
        indicadores=normalizeData(pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters]).rolling(freq).mean()
    else:
        indicadores=pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters].rolling(freq).mean()
    vmin = math.floor(indicadores.min().min())
    vmax = math.ceil(indicadores.max().max())
    plt.figure(figsize=(14,5))
    plt.subplot()
    plt.ylim(vmin, vmax)
    if opcion ==0:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
        for i in indicadores:
            plt.plot(indicadores.loc[:,i],label=str(i), marker='', linewidth=1.5)
        col = math.ceil(len(indicadores)/colEtiquetas)
        plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(indicadores)/col), mode="expand", borderaxespad=0)
    if opcion ==1:
        plt.xlabel("Year", fontsize=font_size, **hfont)
        plt.ylabel("Average Normalized Value", fontsize=font_size, **hfont)
        plt.plot(indicadores.mean(axis=1),label="media", marker='', linewidth=1.5)   
        plt.legend(bbox_to_anchor=(0.01, -0.01, 1, 1), loc='upper left', ncol=1, borderaxespad=0)
    plt.grid(True)   
    plt.title(("Rolling Average "+str(title)).title(), fontsize=font_size, **hfont)
    plt.show()

###########################rolling Avergae #################################
urlFile = 'BOL.xlsx'
sheet_name = 'imp'

#ingresar el cluster a analizar
cluster = 0 
#_namesInClusters = clusters[list(clusters.keys())[cluster-1]]

#Arreglo de indicadores si no se utiliza cluster
_namesInClusters = ["NY.TRF.NCTR.KN","NE.CON.GOVT.KD.ZG"]

colEtiquetas = 5
normalizacion=True
#opcion = 0: rolling averange de los indicadores, opcion=1: rolling average del promedio
plotRollingAverage(urlFile, sheet_name, _namesInClusters, cluster, colEtiquetas, normalizacion, opcion = 0, freq=5)

In [ ]:
def plotTendencyAndCicle(urlFile, sheet_name, _namesInClusters, title, colEtiquetas, normalizacion=True, opcion = 0):
    xlsx = excelFile(urlFile)
    font_size = 16
    hfont = {'fontname':'Arial'}
    if normalizacion:
        indicadores=normalizeData(pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters])
    else:
        indicadores=pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters]
    ciclo, tend = sm.tsa.filters.hpfilter(indicadores, lamb=6.25)
    plt.figure(figsize=(14,5))
    plt.subplot()
    desc = None
    if opcion ==0:
        desc = tend
        plt.title(("Trend").title(), fontsize=font_size, **hfont)
    if opcion ==1:
        desc = ciclo
        plt.title(("Cycle").title(), fontsize=font_size, **hfont)

        
    plt.xlabel("Year", fontsize=font_size, **hfont)
    plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
    vmin = math.floor(desc.min().min())
    vmax = math.ceil(desc.max().max())
    plt.ylim(vmin, vmax)
    for i in desc:
        plt.plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
    col = math.ceil(len(desc)/colEtiquetas)
    plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(desc)/col), mode="expand", borderaxespad=0)
    plt.grid(True)   
    plt.show()

#######################tendencia y ciclo ##########################################
urlFile = 'BOL.xlsx'
sheet_name = 'imp'

#ingresar el cluster a analizar
cluster = 0 
#_namesInClusters = clusters[list(clusters.keys())[cluster-1]]

#Arreglo de indicadores si no se utiliza cluster
_namesInClusters = ["NY.TRF.NCTR.KN","NE.CON.GOVT.KD.ZG"]
colEtiquetas = 5
normalizacion=True
plotTendencyAndCicle(urlFile, sheet_name, _namesInClusters, cluster, colEtiquetas, normalizacion, opcion = 0)

## Descompocicion de series para un input de indicadores, segun el modelo ARIMA

In [ ]:
#freq: 5 para años y 1600 para dias
def plotDescomposition(urlFile, sheet_name, _namesInClusters, title, colEtiquetas, normalizacion=True, freq=5):
    xlsx = excelFile(urlFile)
    font_size = 16
    hfont = {'fontname':'Arial'}
    if normalizacion:
        indicadores=normalizeData(pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters])
    else:
        indicadores=pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters]
    descomposicion = sm.tsa.seasonal_decompose(indicadores.interpolate(),model='additive', freq=freq, two_sided=False) 
    fig, axs = plt.subplots(2, 2, figsize=(14,10))
    fig.suptitle('Decomposition of Cluster {}'.format(title), fontsize=font_size, **hfont)
    
    desc = descomposicion.observed
    for i in desc:
        axs[0, 0].plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
    axs[0, 0].set_title(("Cluster").title(), fontsize=font_size, **hfont)
    col = math.ceil(len(desc)/colEtiquetas)
    axs[0, 0].grid(True)
    
    desc = descomposicion.trend
    for i in desc:
        axs[0, 1].plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
    axs[0, 1].set_title(("Rolling Average").title(), fontsize=font_size, **hfont)
    col = math.ceil(len(desc)/colEtiquetas)
    axs[0, 1].grid(True)
    
    desc = descomposicion.seasonal
    for i in desc:
        axs[1, 0].plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
    axs[1, 0].set_title(("Seasonal").title(), fontsize=font_size, **hfont)
    col = math.ceil(len(desc)/colEtiquetas)
    axs[1, 0].grid(True)
    
    desc = descomposicion.resid
    for i in desc:
        axs[1, 1].plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
    axs[1, 1].set_title(("Residue").title(), fontsize=font_size, **hfont)
    col = math.ceil(len(desc)/colEtiquetas)
    axs[1, 1].grid(True)
    fig.legend(bbox_to_anchor=(0.0, 0.10, 1, 1), loc='upper center', ncol=math.ceil(len(desc)/col), mode="expand", borderaxespad=0)
    fig.subplots_adjust(hspace=0.3)
    for ax in axs.flat:
        ax.set(xlabel='Year', ylabel='Normalized Value')
        
urlFile = 'BOL.xlsx'
sheet_name = 'imp'

#ingresar el cluster a analizar (diccionario)
cluster = 0 
#_namesInClusters = clusters[list(clusters.keys())[cluster-1]]

#Arreglo de indicadores si no se utiliza cluster
_namesInClusters = ["NY.TRF.NCTR.KN","NE.CON.GOVT.KD.ZG"]
colEtiquetas = 5
normalizacion=True
plotDescomposition(urlFile, sheet_name, _namesInClusters, cluster, colEtiquetas, freq=5)

In [ ]:
###########################################################END#############################################################

#### Reporte numero de indicadores afectados implicitamente

In [ ]:
#devuelve el numero de indicadores a los que representa un cluster
def reporteClusterSectors(urlFile, _namesInClusters, nameMostRepresentative, pais):
    xlsx = excelFile(urlFile)
    indicator = pd.read_excel(xlsx, sheet_name="{0}".format(_namesInClusters[0])).set_index("year").columns
    sector = [_namesInClusters[0]]*len(indicator)
    for data in range(1,len(_namesInClusters)):
        aux=pd.read_excel(xlsx, sheet_name="{0}".format(_namesInClusters[data])).set_index("year").columns
        sector = np.concatenate([sector,[_namesInClusters[data]]*len(aux)], axis=0)
        indicator = np.concatenate([indicator,aux],axis=0)  
    rep = readFile(nameMostRepresentative, sheet=pais)
    ind_sec = [[ind, sec] for ind, sec in zip(indicator, sector) if ind in rep.columns ]
    indicator,sector =zip(*ind_sec)
    indicadores = len(rep)-rep.loc[:,indicator].isna().sum()
    indicadores = pd.DataFrame(indicadores, columns=["Nro. subindicadores"])
    indicadores.insert(loc=0, column='Sector', value=list(sector)) 
    indicadores = indicadores.sort_values(by="Nro. subindicadores", ascending=False)
    total = indicadores.iloc[:,1].sum()
    indicadores.loc["Total"] = [None,total]
    indicadores.index.name="Indicadores"
    return  indicadores
#urlFile = 'sector_BRANORM.xlsx'
#cluster = 2
#_namesInClusters = [i for i in clusters[list(clusters.keys())[cluster-1]]]
#nameMostRepresentative = "reporteMostRepresentative_prob0.75.xlsx"
#pais = "BRA"
#a = reporteClusterSectors(urlFile, _namesInClusters, nameMostRepresentative, pais)
#saveFile(a, "{}_r.xlsx".format(pais))

In [ ]:
#devuelve el numero de indicadores a los que representa un cluster
def reporteClusterIndicators(urlFile, sheet_name, _namesInClusters, nameMostRepresentative):
    xlsx = excelFile(urlFile)
    indicator = pd.read_excel(xlsx, sheet_name=sheet_name).set_index("year").loc[:,_namesInClusters].columns
    rep = readFile(nameMostRepresentative, sheet=sheet_name)
    indicadores = len(rep)-rep.loc[:,indicator].isna().sum()
    indicadores = pd.DataFrame(indicadores, columns=["Nro. subindicadores"])
    indicadores = indicadores.sort_values(by="Nro. subindicadores", ascending=False)
    total = indicadores.iloc[:,0].sum()
    indicadores.loc["Total"] = [total]
    indicadores.index.name="Indicadores"
    return  indicadores
urlFile = 'imputacionCHLNORM.xlsx'
cluster = 2
_namesInClusters = clusters[list(clusters.keys())[cluster-1]]
nameMostRepresentative = "reporteMostRepresentative_prob0.75.xlsx"
sheet_name = "CHL"
a = reporteClusterIndicators(urlFile, sheet_name, _namesInClusters, nameMostRepresentative)
saveFile(a, "{}_r.xlsx".format(sheet_name))

#### Reporte sector 

In [ ]:
# recuperarIndicadorPorSector1:Recuperamos los indicadores de un sector y como se han distribuido atravez de los otros sectores
#PARAMS:
#_frameMostRepresentative = dataFrame reporteMostRepresentative_prob0.75.xlsx
#_frameSectores = dtaFrame seriesCETS.csv
#_framePais = dataFRame imputacionCHLNORM.xlsx
#_agrupacion = "Topic" dato mediante el cual realiza las agrupaciones en _frameSectores
#_sector = "gender" sector de la cual se quiere ver como se ha distribuido en los otros indicadores
def ubicarSubindicadores(mostRepresentative, indicators):
    bools = {}
    for ind in mostRepresentative:
        aux = list(mostRepresentative[ind][mostRepresentative[ind].isin(indicators)])
        if len(aux)>0:
            bools[ind]=aux
    return bools

def buscarIndicadores1(_dataFrameColumns,_indicadores):
    indexs = []
    for index in _dataFrameColumns:
        val = np.where(index==_indicadores)[0]
        if val!=None:
            indexs.append(list(val)[0])
    return _indicadores[indexs]

def recuperarIndicadorPorSector1(_frameMostRepresentative, _frameSectores, _framePais, _agrupacion, _sector):
    with pd.ExcelWriter('datos_reales/spread_{0}.xlsx'.format(_sector)) as writer:
        sectores = _frameSectores.groupby(_agrupacion)
        indicadores = None
        for index, group in sectores:
            if index.lower() == _sector:
                indicadores =ubicarSubindicadores(_frameMostRepresentative,np.array(group.iloc[:,0]))
        aux_df = pd.DataFrame()
        for index, group in sectores:
            grupo = buscarIndicadores1(indicadores.keys(), np.array(group.iloc[:,0]))
            aux_grupo = {}
            maxs = 0
            if len(grupo)>0:
                for i in grupo:
                    if maxs<len(indicadores[i]):
                        maxs = len(indicadores[i])
                    aux_grupo[i]=indicadores[i]
                for key,values in aux_grupo.items():    
                    array = values
                    end = [None] * (maxs - len(array))
                    if len(array) <= maxs:
                        array = np.concatenate([array,end])
                    aux_grupo[key]=array
            
                df = pd.DataFrame(aux_grupo)
                df.columns = pd.MultiIndex.from_product([[str(index).lower()],df.columns])
                
                if aux_df.empty:
                    aux_df = df
                else:
                    aux_df = pd.concat([aux_df,df], axis=1)
        aux_df.T.to_excel(writer, sheet_name=_sector, index=True)
_frameMostRepresentative = readFile('reporteMostRepresentative_prob0.75.xlsx', sheet="BRA")           
_frameSectores = readFile('seriesCETS.csv')
_framePais = readFile("imputacionBRANORM.xlsx", "BRA").set_index("year")
_agrupacion = "Topic"
_sector = "environment"
recuperarIndicadorPorSector1(_frameMostRepresentative, _frameSectores, _framePais, _agrupacion, _sector)

In [ ]:
#CODIGO POR REVISAR
series = readFile("wdi_0.1.xlsx","Data").groupby(by="Country Code")
indicators = ["SE.SEC.AGES", "SE.PRM.DURS", "SE.PRM.ENRR","SE.PRM.ENRR.FE", "SE.PRM.ENRR.MA", "SE.SEC.DURS"]
series = series.get_group("ARG").reset_index()
series = series.iloc[series["Indicator Code"][series["Indicator Code"].isin(indicators)].index, 37:60]
font_size = 16
hfont = {'fontname':'Arial'}
plt.figure(figsize=(15,5))
ciclo, tend = sm.tsa.filters.hpfilter(normalizeData(series.T.interpolate()), lamb=6.25)
plt.subplot()
desc = None
opcion = 0
if opcion ==0:
    desc = tend
    plt.title(("Trend").title(), fontsize=font_size, **hfont)
if opcion ==1:
    desc = ciclo
    plt.title(("Cycle").title(), fontsize=font_size, **hfont)

        
plt.xlabel("Year", fontsize=font_size, **hfont)
plt.ylabel("Normalized Value", fontsize=font_size, **hfont)
vmin = math.floor(desc.min().min())
vmax = math.ceil(desc.max().max())
plt.ylim(vmin, vmax)
for i in desc:
    plt.plot(desc.loc[:,i],label=str(i), marker='', linewidth=1.5)
col = math.ceil(len(desc)/colEtiquetas)
plt.legend(bbox_to_anchor=(0.0, 0.29, 1, 1), loc='upper center', ncol=math.ceil(len(desc)/col), mode="expand", borderaxespad=0)
plt.grid(True)   
plt.show()
#
#series = series.iloc[series["Indicator Code"][series["Indicator Code"].isin(indicators)].index, 36:60]
#series